# IMPORT LIBRARIES

In [1]:
import cobra
import glob
import os
import time
import pandas as pd

# RECON 3D

In [2]:
recon3D = cobra.io.load_matlab_model('/Users/claudiavicentecomorera/Desktop/TFG/Data/Recon3D_301/Recon3DModel_301.mat')
recon3D

No defined compartments in model Recon3DModel. Compartments will be deduced heuristically using regular expressions.
Using regular expression found the following compartments:c, e, g, i, l, m, n, r, x


Name,Recon3DModel
Memory address,1266acad0
Number of metabolites,5835
Number of reactions,10600
Number of genes,2248
Number of groups,103
Objective expression,1.0*biomass_maintenance - 1.0*biomass_maintenance_reverse_95d2f
Compartments,"c, l, m, r, e, x, n, g, i"


In [3]:
solution_recon_3D = recon3D.optimize()
bm = solution_recon_3D.objective_value
solution_recon_3D

,fluxes,reduced_costs
10FTHF5GLUtl,0.0,0.000000e+00
10FTHF5GLUtm,0.0,-8.031949e-02
10FTHF6GLUtl,0.0,0.000000e+00
10FTHF6GLUtm,0.0,0.000000e+00
10FTHF7GLUtl,0.0,0.000000e+00
...,...,...
CYOR_u10mi,500.0,0.000000e+00
Htmi,1000.0,-7.246382e-19
NADH2_u10mi,0.0,7.145600e-18
CYOOm3i,750.0,-3.009266e-33


# FLUXES SIMULATION: rFASTCORMICS EXAMPLE_V4

In [4]:
model1 =  cobra.io.load_matlab_model("/Users/claudiavicentecomorera/Desktop/TFG/rFASTCORMICS/rFASTCORMICS_RNA-seq/exampleResults/Example_v4_3D/SampleModel_TCGA06067511A32RA36H07.mat")
model2 =  cobra.io.load_matlab_model("/Users/claudiavicentecomorera/Desktop/TFG/rFASTCORMICS/rFASTCORMICS_RNA-seq/exampleResults/Example_v4_3D/SampleModel_TCGA06067811A32RA36H07.mat")

No defined compartments in model ContextModel. Compartments will be deduced heuristically using regular expressions.
Using regular expression found the following compartments:c, e, g, i, l, m, n, r, x
No defined compartments in model ContextModel. Compartments will be deduced heuristically using regular expressions.
Using regular expression found the following compartments:c, e, g, i, l, m, n, r, x


In [5]:
model1

Name,ContextModel
Memory address,152d58490
Number of metabolites,2522
Number of reactions,3412
Number of genes,2248
Number of groups,87
Objective expression,1.0*biomass_maintenance - 1.0*biomass_maintenance_reverse_95d2f
Compartments,"c, m, x, e, l, r, g, n, i"


In [6]:
model2

Name,ContextModel
Memory address,145544950
Number of metabolites,2545
Number of reactions,3448
Number of genes,2248
Number of groups,88
Objective expression,1.0*biomass_maintenance - 1.0*biomass_maintenance_reverse_95d2f
Compartments,"c, l, m, x, e, r, g, n, i"


In [7]:
t0_1 = time.time()  
solution1 = model1.optimize()
t_1 = time.time()  
time1 = t_1 - t0_1
print(f"The simulation of {model1.id} fluxes takes: {time1} seconds")

bm1 = solution1.objective_value
solution1

The simulation of ContextModel fluxes takes: 0.6805949211120605 seconds


,fluxes,reduced_costs
13DAMPPOX,0.000000,-8.860996e-02
2HBO,0.000000,-3.367852e-19
2HBt2,-103.574491,5.051778e-19
2HCO3_NAt,-374.003830,-0.000000e+00
2OXOADOXm,0.000000,1.572093e-18
...,...,...
ATPS4mi,1000.000000,1.965986e-02
CYOR_u10mi,595.460987,-5.204170e-18
Htmi,0.000000,-5.929563e-04
NADH2_u10mi,0.000000,-7.880809e-03


In [8]:
t0_2 = time.time()  
solution2 = model2.optimize()
t_2 = time.time()  
time2 = t_2 - t0_2
print(f"The simulation of {model2.id} fluxes takes: {time2} seconds")

bm2 = solution2.objective_value
solution2

The simulation of ContextModel fluxes takes: 0.5236332416534424 seconds


,fluxes,reduced_costs
10FTHF7GLUtl,0.000000,-1.734723e-18
10FTHFtl,0.000000,-0.000000e+00
13DAMPPOX,0.000000,-4.999028e-19
2AMACHYD,0.000000,1.234821e-18
2HBO,0.000000,-1.683926e-19
...,...,...
EX_dolichol_L[e],0.000000,0.000000e+00
ATPS4mi,1000.000000,1.846234e-02
CYOR_u10mi,495.730221,-5.204170e-18
NADH2_u10mi,0.000000,-1.360383e-02


In [9]:
print((bm1/bm)*100)
print((bm2/bm)*100)

8.398782634279042
8.17256017914707


# FLUXES SIMULATION: rFASTCORMICS ADDITIONAL EXAMPLE

In [10]:
f_path = "/Users/claudiavicentecomorera/Desktop/TFG/rFASTCORMICS/rFASTCORMICS_RNA-seq/exampleResults/Example_BRCA_3D/"
file_paths = glob.glob(f_path + "*.mat")

models = []
solutions = []
times = []

for file_path in file_paths:
    model = cobra.io.load_matlab_model(file_path)
    model.id = os.path.splitext(os.path.basename(file_path))[0]

    t0_add = time.time()  
    solution = model.optimize()
    t_add = time.time()  
    time_add = t_add - t0_add 
    
    print(f"The simulation of {model.id} fluxes takes: {time_add} seconds")
    
    models.append(model)
    solutions.append(solution)
    times.append(time_add)

No defined compartments in model ContextModel. Compartments will be deduced heuristically using regular expressions.
Using regular expression found the following compartments:c, e, g, i, l, m, n, r, x
No defined compartments in model ContextModel. Compartments will be deduced heuristically using regular expressions.
Using regular expression found the following compartments:c, e, g, i, l, m, n, r, x


The simulation of SampleModel_BRCA_C_TCGA_E9_A1NC_01A_21R_A26B_07 fluxes takes: 0.686474084854126 seconds


No defined compartments in model ContextModel. Compartments will be deduced heuristically using regular expressions.
Using regular expression found the following compartments:c, e, g, i, l, m, n, r, x


The simulation of SampleModel_BRCA_C_TCGA_AO_A12H_01A_11R_A115_07 fluxes takes: 0.6226742267608643 seconds


No defined compartments in model ContextModel. Compartments will be deduced heuristically using regular expressions.
Using regular expression found the following compartments:c, e, g, i, l, m, n, r, x


...

The simulation of SampleModel_BRCA_C_TCGA_D8_A4Z1_01A_21R_A266_07 fluxes takes: 0.6689767837524414 seconds


No defined compartments in model ContextModel. Compartments will be deduced heuristically using regular expressions.
Using regular expression found the following compartments:c, e, g, i, l, m, n, r, x


The simulation of SampleModel_BRCA_H_TCGA_BH_A204_11A_53R_A157_07 fluxes takes: 0.6936891078948975 seconds


No defined compartments in model ContextModel. Compartments will be deduced heuristically using regular expressions.
Using regular expression found the following compartments:c, e, g, i, l, m, n, r, x


The simulation of SampleModel_BRCA_H_TCGA_BH_A0DH_11A_31R_A089_07 fluxes takes: 0.6667792797088623 seconds
The simulation of SampleModel_BRCA_C_TCGA_A1_A0SM_01A_11R_A084_07 fluxes takes: 0.7247421741485596 seconds


In [11]:
solutions[1]

,fluxes,reduced_costs
10FTHF7GLUtl,0.000000,-4.398388e-02
10FTHF7GLUtm,0.000000,-6.938894e-18
10FTHFtl,0.000000,-0.000000e+00
13DAMPPOX,0.000000,-7.952625e-18
2HBO,0.000000,-0.000000e+00
...,...,...
DM_Lkynr[c],0.000000,0.000000e+00
ATPS4mi,1000.000000,4.398388e-02
CYOR_u10mi,507.759152,0.000000e+00
Htmi,0.000000,-1.140484e-17


In [12]:
r_models = pd.DataFrame(index=[model.id for model in models])

for i, solution in enumerate(solutions, start = 0):
    bm_m = solution.objective_value
    percent = (bm_m/bm) * 100
    
    r_models.loc[r_models.index[i], 'flux_bm'] = bm_m
    r_models.loc[r_models.index[i], 'percent_bm'] = percent
    r_models.loc[r_models.index[i], 'feasible_models'] = "TRUE" if percent > 5 else "FALSE"

print(f'The 5% of Recon 2 v.04 objective value is {5/100*bm}')
print(f'There are {r_models["feasible_models"].eq("FALSE").sum()} models that aren\'t feasible because of their low objective value.\n')
r_models

The 5% of Recon 2 v.04 objective value is 37.750160777533154
There are 0 models that aren't feasible because of their low objective value.



,flux_bm,percent_bm,feasible_models
SampleModel_BRCA_C_TCGA_E9_A1NC_01A_21R_A26B_07,74.087590,9.812884,TRUE
SampleModel_BRCA_C_TCGA_AO_A12H_01A_11R_A115_07,87.967757,11.651309,TRUE
SampleModel_BRCA_C_TCGA_B6_A0RH_01A_21R_A115_07,78.912376,10.451926,TRUE
SampleModel_BRCA_H_TCGA_BH_A1EO_11A_31R_A137_07,70.695423,9.363592,TRUE
SampleModel_BRCA_H_TCGA_BH_A1FG_11B_12R_A13Q_07,51.622864,6.837436,TRUE
SampleModel_BRCA_C_TCGA_A8_A0AB_01A_11R_A034_07,52.926298,7.010076,TRUE
SampleModel_BRCA_H_TCGA_BH_A0BC_11A_22R_A089_07,58.815409,7.790087,TRUE
SampleModel_BRCA_H_TCGA_BH_A0DT_11A_12R_A12D_07,69.641152,9.223954,TRUE
SampleModel_BRCA_H_TCGA_BH_A0BA_11A_22R_A19E_07,72.390478,9.588102,TRUE
SampleModel_BRCA_C_TCGA_A2_A0D0_01A_11R_A00Z_07,56.281054,7.454412,TRUE
